In [2]:
import pandas as pd

In [10]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [11]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [12]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [13]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords

#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

/home/mr-rider/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [10]:
#from nltk.corpus import stopwords

In [14]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [15]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [16]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [17]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


CPU times: user 1min 20s, sys: 1.58 s, total: 1min 22s
Wall time: 1min 22s


In [18]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 8min 55s, sys: 873 ms, total: 8min 56s
Wall time: 8min 59s


In [19]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [20]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 3min, sys: 1min 44s, total: 4min 45s
Wall time: 1min 41s


In [21]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [22]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(0, 0.4592455), (8, 0.49169537), (10, 0.025558036)]

In [23]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: украина всё новый проблема первый россия украинский
topic_1: экономика ребёнок всё жизнь первый nn школа
topic_2: ступень балтийский профессия физика комплексный спектакль лауреат
topic_3: исследование гражданин население nn лаборатория доказательство белоруссия
topic_4: сша россия российский американский nn северный операция
topic_5: военный россия армия район гражданский nn российский
topic_6: университет журнал рекомендовать студент виза бомба образовать
topic_7: млрд рубль банк россия фонд млн nn
topic_8: nn фестиваль первый земля сайт увидеть день
topic_9: ракета китай китайский высота рейтинг станция место
topic_10: nn день праздник доклад офицер фильм участник
topic_11: россия статья правительство новый проект российский nn
topic_12: газ турция турецкий район население чёрный сектор
topic_13: квартира мальчик собственность следствие макаров челябинский медик
topic_14: пострадать земля авария автомобиль район захватить образоваться
topic_15: взрыв погибнуть разместить го

In [24]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [25]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.82517,0.131156,...,0.000000,0.0,0.0,0.0,0.0,0.035262,0.0,0.0,0.0,0.0
1,4896,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.404452,...,0.149341,0.0,0.0,0.0,0.0,0.423032,0.0,0.0,0.0,0.0
2,4897,0.459150,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.491706,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,4898,0.718749,0.014387,0.0,0.0,0.000000,0.0,0.0,0.00000,0.255663,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,4899,0.000000,0.000000,0.0,0.0,0.207523,0.0,0.0,0.00000,0.120262,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### Следующий шаг - векторные представления пользователей

In [31]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

### 2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [1]:
user_articles_list = users['articles'].iloc[33]
#Модифицировать код функции get_user_embedding таким образом, 
# чтобы считалось не среднее (как в примере np.mean), а медиана.

def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

NameError: name 'users' is not defined

In [25]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.0,0.017025,0.0,0.038494,0.021371,0.0,0.000000,0.000000,0.000000,...,0.000000,0.010121,0.000000,0.000000,0.027170,0.000000,0.0,0.061735,0.0,0.000000
1,u108690,0.0,0.000000,0.0,0.081444,0.022936,0.0,0.000000,0.015372,0.007502,...,0.000000,0.000000,0.046902,0.008024,0.178603,0.010911,0.0,0.122660,0.0,0.000000
2,u108339,0.0,0.000000,0.0,0.079962,0.017540,0.0,0.007184,0.008485,0.009613,...,0.010519,0.000000,0.022400,0.032142,0.196408,0.000000,0.0,0.156623,0.0,0.110988


In [34]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [27]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.0,0.017025,0.0,0.038494,0.021371,0.0,0.000000,0.000000,0.000000,...,0.010121,0.000000,0.000000,0.027170,0.000000,0.0,0.061735,0.0,0.000000,0
1,u108690,0.0,0.000000,0.0,0.081444,0.022936,0.0,0.000000,0.015372,0.007502,...,0.000000,0.046902,0.008024,0.178603,0.010911,0.0,0.122660,0.0,0.000000,1
2,u108339,0.0,0.000000,0.0,0.079962,0.017540,0.0,0.007184,0.008485,0.009613,...,0.000000,0.022400,0.032142,0.196408,0.000000,0.0,0.156623,0.0,0.110988,1


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [29]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [30]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [31]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.09430844, 0.0127813 , 0.41403998, 0.19591322, 0.13584548,
       0.02378314, 0.08819   , 0.06818253, 0.0948032 , 0.0549567 ])

In [43]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [33]:
### Рассчитаем Precision, Recall, F_score

In [34]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.272792, F-Score=0.812, Precision=0.772, Recall=0.857


### roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [35]:
roc_auc_score(y_test, preds)

0.9816198616198616

### 3. Повторить п.2, но используя уже не медиану, а max

In [33]:
def get_user_embedding(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [35]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.316939,0.074806,0.051493,0.050838,0.286127,0.604095,0.020276,0.223757,0.338197,...,0.033880,0.030288,0.000000,0.039053,0.020720,0.148449,0.130674,0.259902,0.000000,0.030705
1,u108690,0.405999,0.238475,0.000000,0.018654,0.141602,0.146775,0.029415,0.207381,0.012235,...,0.062177,0.023718,0.019647,0.000000,0.030071,0.124590,0.033347,0.274217,0.000000,0.058407
2,u108339,0.072442,0.094141,0.014127,0.016119,0.277355,0.180129,0.020609,0.365474,0.211170,...,0.027759,0.000000,0.000000,0.043597,0.033697,0.391959,0.053600,0.067453,0.022094,0.048512


In [36]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.316939,0.074806,0.051493,0.050838,0.286127,0.604095,0.020276,0.223757,0.338197,...,0.030288,0.000000,0.039053,0.020720,0.148449,0.130674,0.259902,0.000000,0.030705,0
1,u108690,0.405999,0.238475,0.000000,0.018654,0.141602,0.146775,0.029415,0.207381,0.012235,...,0.023718,0.019647,0.000000,0.030071,0.124590,0.033347,0.274217,0.000000,0.058407,1
2,u108339,0.072442,0.094141,0.014127,0.016119,0.277355,0.180129,0.020609,0.365474,0.211170,...,0.000000,0.000000,0.043597,0.033697,0.391959,0.053600,0.067453,0.022094,0.048512,1


In [39]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [40]:
logreg = LogisticRegression()
#обучим 
logreg.fit(X_train, y_train)

LogisticRegression()

In [41]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

array([0.2408437 , 0.00088243, 0.76438527, 0.21359929, 0.16234637,
       0.01595084, 0.00656865, 0.00419781, 0.00237502, 0.31531617])

In [44]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.317122, F-Score=0.797, Precision=0.771, Recall=0.824


In [45]:
roc_auc_score(y_test, preds)

0.972214663643235

### 5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [46]:
scores = pd.DataFrame({'Type': ['mean', 'median', 'max'], 'roc auc':[0.9507924879353452,0.9816198616198616, 0.972214663643235],
                      'precison': [0.651,0.772,0.758], 'recall':[0.747,0.857,0.820], 
                      'f_score':[0.696,0.812,0.788]})

In [47]:
scores.head()

,Type,roc auc,precison,recall,f_score
0,mean,0.950792,0.651,0.747,0.696
1,median,0.981620,0.772,0.857,0.812
2,max,0.972215,0.758,0.820,0.788


### 6.Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [ ]:
# Наиболее эффективным оказался способ с median. Mean -сильно чувствителен к выбросам. В max попали выбросы.